In [1]:
import sqlite3
from sqlite3 import OperationalError
from IPython.core.display import display, HTML

import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)


# Open/create sqlite database, reset so tables from exercise are dropped.
db = sqlite3.connect('L18DB.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS candidates")
cursor.execute("DROP TABLE IF EXISTS contributors")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE candidates (
               id INTEGER PRIMARY KEY NOT NULL, 
               first_name TEXT, 
               last_name TEXT, 
               middle_init TEXT, 
               party TEXT NOT NULL)''')

cursor.execute('''CREATE TABLE contributors (
          id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
          last_name TEXT, 
          first_name TEXT, 
          middle_name TEXT, 
          street_1 TEXT, 
          street_2 TEXT, 
          city TEXT, 
          state TEXT, 
          zip TEXT, 
          amount REAL, 
          date DATETIME, 
          candidate_id INTEGER NOT NULL, 
          FOREIGN KEY(candidate_id) REFERENCES candidates(id))''')



with open ("candidates.txt") as candidates:
    next(candidates) # jump over the header
    for line in candidates.readlines():
        cid, first_name, last_name, middle_name, party = line.strip().split('|')
        vals_to_insert = (int(cid), first_name, last_name, middle_name, party)
        cursor.execute('''INSERT INTO candidates 
                  (id, first_name, last_name, middle_init, party)
                  VALUES (?, ?, ?, ?, ?)''', vals_to_insert)
        
with open ("contributors.txt") as contributors:
    next(contributors)
    for line in contributors.readlines():
        cid, last_name, first_name, middle_name, street_1, street_2, \
            city, state, zip_code, amount, date, candidate_id = line.strip().split('|')
        vals_to_insert = (last_name, first_name, middle_name, street_1, street_2, 
                          city, state, int(zip_code), amount, date, candidate_id)
        cursor.execute('''INSERT INTO contributors (last_name, first_name, middle_name, 
                           street_1, street_2, city, state, zip, amount, date, candidate_id) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert) 


db.commit()


def viz_tables(cols, query, print_query=True):
    """Prints SQL query and displays DataFrame created from queried records."""
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    if print_query:
        print(query)
    display(pd.DataFrame.from_items(framelist))

    
def get_table_columns(table_name):
    """Returns list of column names in given table."""
    query = "PRAGMA table_info({0})".format(table_name)
    return [col[1] for col in cursor.execute(query)]


# Get list of column names for each table.
candidate_cols = get_table_columns('candidates')
contributor_cols = get_table_columns('contributors')


def print_section_break(section, style='h1'):
    print('='*40)
    display(HTML('<{0}>{1}</{0}>'.format(style, section)))
    print('='*40)
    
def print_update(update, style='h4'):
    print('')
    display(HTML('<{0}>{1}</{0}>'.format(style, update)))
    print('')
    
print_section_break('Step 2: Various Queries')

query1 = "SELECT * FROM contributors WHERE state = 'PA'"
viz_tables(contributor_cols, query1)

query2 = "SELECT * FROM contributors WHERE amount > 1000"
viz_tables(contributor_cols, query2)

query3 = "SELECT * FROM contributors WHERE state = 'UT' AND amount > 1000"
viz_tables(contributor_cols, query3)

query4 = "SELECT * FROM contributors WHERE state = ''"
viz_tables(contributor_cols, query4)

query5 = "SELECT * FROM contributors WHERE state IN ('WA', 'PA')"
viz_tables(contributor_cols, query5)

query6 = "SELECT * FROM contributors WHERE amount BETWEEN 100 AND 200"
viz_tables(contributor_cols, query6)

print_section_break('Step 3: Sorting')

query7 = "SELECT * FROM contributors ORDER BY last_name DESC"
viz_tables(candidate_cols, query7)

query8 = "SELECT * FROM contributors WHERE amount BETWEEN 1000 AND 5000 ORDER BY amount DESC"
viz_tables(contributor_cols, query8)

query9 = "SELECT * FROM contributors WHERE amount BETWEEN 1000 AND 5000 ORDER BY candidate_id DESC, amount DESC"
viz_tables(contributor_cols, query9)

print_section_break('Step 4: Selecting Columns')

# Selecting unique combinations of last_name and first_name i.e., some rows 
# may have duplicate values for either column but no rows will have the same 
# last_name, first_name pair.
query10 = "SELECT DISTINCT last_name, first_name FROM contributors"
viz_tables(['last_name', 'first_name'], query10)

print_section_break('Step 5: Altering Tables')

print_update('Adding column full_name to candidates table.')
# Alter tables.
try:
    cursor.execute('''ALTER TABLE candidates ADD COLUMN full_name TEXT''')
except OperationalError:
    print('Error adding full_name column to candidates. May already exist.')

candidate_cols = get_table_columns('candidates')

# Create list of tuples: (full_name, id)
query = '''SELECT id, last_name, first_name FROM candidates'''
full_name_and_id = [(attr[1] + ", " + attr[2], attr[0]) 
                    for attr in cursor.execute(query).fetchall()] 

# Update full_name column with tuples created above.
update = '''UPDATE candidates SET full_name = ? WHERE id = ?''' 
for rows in full_name_and_id:
    cursor.execute(update, rows)

viz_tables(candidate_cols, '''SELECT * FROM candidates''', False)

update = '''UPDATE candidates SET full_name = "Eventual Winner" WHERE last_name = "Obama"'''
cursor.execute(update)
update = '''UPDATE candidates SET full_name = "Eventual Loser" WHERE last_name = "McCain"'''
cursor.execute(update)

print_update('Changed full_name for last_name Obama and McCain.')
viz_tables(candidate_cols, "SELECT * FROM candidates")

print_update('Adding column full_name to contributors table.')
try:
    cursor.execute('''ALTER TABLE contributors ADD COLUMN full_name TEXT''')
except OperationalError:
    print('Error adding full_name column to contributors. May already exist.')

cursor.execute("UPDATE contributors SET full_name = 'Too Much' WHERE amount > 1000")
viz_tables(['full_name', 'amount'], "SELECT * FROM contributors")

contributor_cols = get_table_columns('contributors')

print_section_break('Step 6: Aggregation')

agg_query1 = 'SELECT COUNT(amount) FROM contributors WHERE amount > 1000'
viz_tables(['amount'], agg_query1)

agg_query2 = 'SELECT AVG(amount) FROM contributors'
viz_tables(['amount'], agg_query2)

agg_query3= 'SELECT state, AVG(amount) FROM contributors GROUP BY state'
viz_tables(['state', 'amount'], agg_query3)

print_section_break('Step 7: DELETE')

deletion1 = "DELETE FROM contributors WHERE last_name = 'Ahrens'"
viz_tables(contributor_cols, deletion1)

print_section_break('Step 8: LIMIT')
 
# 10 most generous donors.
limit1 = """
SELECT id, first_name, last_name, SUM(amount) AS total_donations 
FROM contributors 
GROUP BY id 
ORDER BY total_donations DESC 
LIMIT 10
"""
viz_tables(['id', 'first_name', 'last_name', 'total_donations'], limit1)

# 10 least generous donors (with positive amount).
limit2 = """
SELECT id, first_name, last_name, SUM(amount) AS total_donations 
FROM contributors 
GROUP BY id 
HAVING total_donations > 0 
ORDER BY total_donations ASC
LIMIT 10
"""
viz_tables(['id', 'first_name', 'last_name', 'total_donations'], limit2)

db.commit()
db.close()

SELECT * FROM contributors WHERE state = 'PA'


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,71,BUCKLEY,WALTER,W.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
1,72,BUCKLEY,MARJORIE,B.,1635 COUNTRY ROAD,,BETHLEHEM,PA,180155718,-100.0,2008-03-05,22
2,94,Raught,Philip,M,4714 Plum Way,,Pittsburgh,PA,15201,-1046.0,2008-04-21,32
3,95,Ferrara,Judith,D,1508 Waterford Road,,Yardley,PA,19067,-1100.0,2008-04-21,32
4,166,ABEL,JOHN,H.,422 THOMAS STREET,,BETHLEHEM,PA,180153316,200.0,2008-01-22,37


SELECT * FROM contributors WHERE amount > 1000


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,6,Akin,Mike,,181 Baywood Lane,,Monticello,AR,71655,1500.0,2007-05-18,16
1,10,Allen,John D.,,1052 Cannon Mill Drive,,North Augusta,SC,29860,1300.0,2007-06-29,16
2,14,Altes,R.D.,,8600 Moody Road,,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,,211 Long Island Drive,,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,,2550 Adamsbrooke Drive,,Conway,AR,72034,2300.0,2007-04-11,16
5,29,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,2300.0,2007-08-14,20
6,31,Buckel,Linda,,PO Box 683130,,Park City,UT,840683130,4600.0,2007-08-14,20
7,34,Buck,Blaine,M,45 Eaton Ave,,Camden,ME,48431752,2300.0,2007-09-30,20
8,46,Buchanan,John,,2025 NW 29th Rd,,Boca Raton,FL,334316303,1300.0,2007-08-09,20
9,136,ABRAMOWITZ,NIRA,,411 HARBOR ROAD,,SOUTHPORT,CT,68901376,2300.0,2007-09-14,35


SELECT * FROM contributors WHERE state = 'UT' AND amount > 1000


...

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id,full_name



SELECT id, first_name, last_name, SUM(amount) AS total_donations 
FROM contributors 
GROUP BY id 
ORDER BY total_donations DESC 
LIMIT 10



,id,first_name,last_name,total_donations
0,31,Linda,Buckel,4600.0
1,160,MARIA,ABATE,2600.0
2,14,R.D.,Altes,2300.0
3,16,John,Anthony,2300.0
4,22,David,Baker,2300.0
5,29,Linda,Buckel,2300.0
6,34,Blaine,Buck,2300.0
7,136,NIRA,ABRAMOWITZ,2300.0
8,6,Mike,Akin,1500.0
9,10,John D.,Allen,1300.0



SELECT id, first_name, last_name, SUM(amount) AS total_donations 
FROM contributors 
GROUP BY id 
HAVING total_donations > 0 
ORDER BY total_donations ASC
LIMIT 10



,id,first_name,last_name,total_donations
0,27,Steve,Buckler,25.0
1,50,Ryan,Harrison,25.0
2,102,Elaine,Aarons,25.0
3,141,PATRICIA,ABEGG,25.0
4,144,PATRICIA,ABEGG,25.0
5,145,PATRICIA,ABEGG,25.0
6,161,PETER,ABAIR,25.0
7,159,PAULINE,ABBO,35.0
8,19,William,Ardle,50.0
9,26,Steve,Buckler,50.0
